# Analiza orzeczeń sądu najwyższego

In [1]:
import spacy 
import pandas as pd
import numpy as np
import os
os.chdir('..')

In [47]:
dflist = []
for i in range(0,5):
    df = pd.read_csv('./resources/supreme-court_chunk{}.zip'.format(i))
    dflist.append(df)

df = pd.concat(dflist)

In [3]:
df = df[df["judgmenttype"] == 'SENTENCE'] 
df['text'] = df.textcontent.replace(r'\n',' ', regex=True) 
df['text'] = df['text'].str.lower()
df['uzasadnienie'] = df['text'].str.extract('(uzasadnienie[^>]*)', expand=True)
df['appealSuccesful'] = df.textcontent.str.contains(r'(uchyla zaskarżony)|(uchyla zaskarżone)', regex=True)
df.groupby(['appealSuccesful']).size()

c:\Users\Mateu\anaconda3\lib\site-packages\pandas\core\strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


appealSuccesful
False    12207
True      5978
dtype: int64

In [13]:
sample_records = df.head(10)
sample_records.to_csv('./resources/sample_supreme.csv')

In [4]:
df['contWithoutTarget'] = df['uzasadnienie'].str.replace(r'(uchyla zaskarżony)|(uchyla zaskarżone)', '', regex=True)
nlp = spacy.load('pl_core_news_sm')

df["lemmatized"] = df['contWithoutTarget'].apply(lambda x: " ".join([y.lemma_ for y in nlp(str(x))]))
X = df['lemmatized']
df['target'] = np.where(df['appealSuccesful']==True, 'success', 'failure')
stopwords = ['protokolant','sędzia','ssa', 'spr','a','aby','ach','acz','aczkolwiek','aj','albo','ale','alez','ależ','ani','az','aż','bardziej','bardzo','beda','bedzie','bez','deda','będą','bede','będę','będzie','bo','bowiem','by','byc','być','byl','byla','byli','bylo','byly','był','była','było','były','bynajmniej','cala','cali','caly','cała','cały','ci','cie','ciebie','cię','co','cokolwiek','cos','coś','czasami','czasem','czemu','czy','czyli','daleko','dla','dlaczego','dlatego','do','dobrze','dokad','dokąd','dosc','dość','duzo','dużo','dwa','dwaj','dwie','dwoje','dzis','dzisiaj','dziś','gdy','gdyby','gdyz','gdyż','gdzie','gdziekolwiek','gdzies','gdzieś','go','i','ich','ile','im','inna','inne','inny','innych','iz','iż','ja','jak','jakas','jakaś','jakby','jaki','jakichs','jakichś','jakie','jakis','jakiś','jakiz','jakiż','jakkolwiek','jako','jakos','jakoś','ją','je','jeden','jedna','jednak','jednakze','jednakże','jedno','jego','jej','jemu','jesli','jest','jestem','jeszcze','jeśli','jezeli','jeżeli','juz','już','kazdy','każdy','kiedy','kilka','kims','kimś','kto','ktokolwiek','ktora','ktore','ktorego','ktorej','ktory','ktorych','ktorym','ktorzy','ktos','ktoś','która','które','którego','której','który','których','którym','którzy','ku','lat','lecz','lub','ma','mają','mało','mam','mi','miedzy','między','mimo','mna','mną','mnie','moga','mogą','moi','moim','moj','moja','moje','moze','mozliwe','mozna','może','możliwe','można','mój','mu','musi','my','na','nad','nam','nami','nas','nasi','nasz','nasza','nasze','naszego','naszych','natomiast','natychmiast','nawet','nia','nią','nic','nich','nie','niech','niego','niej','niemu','nigdy','nim','nimi','niz','niż','no','o','obok','od','około','on','ona','one','oni','ono','oraz','oto','owszem','pan','pana','pani','po','pod','podczas','pomimo','ponad','poniewaz','ponieważ','powinien','powinna','powinni','powinno','poza','prawie','przeciez','przecież','przed','przede','przedtem','przez','przy','roku','rowniez','również','sam','sama','są','sie','się','skad','skąd','soba','sobą','sobie','sposob','sposób','swoje','ta','tak','taka','taki','takie','takze','także','tam','te','tego','tej','ten','teraz','też','to','toba','tobą','tobie','totez','toteż','totobą','trzeba','tu','tutaj','twoi','twoim','twoj','twoja','twoje','twój','twym','ty','tych','tylko','tym','u','w','wam','wami','was','wasz','wasza','wasze','we','według','wiele','wielu','więc','więcej','wlasnie','właśnie','wszyscy','wszystkich','wszystkie','wszystkim','wszystko','wtedy','wy','z','za','zaden','zadna','zadne','zadnych','zapewne','zawsze','ze','zeby','zeznowu','zł','znow','znowu','znów','zostal','został','żaden','żadna','żadne','żadnych','że','żeby']

In [10]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

ylabels = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2, random_state=42)
bow = CountVectorizer(stop_words=stopwords, ngram_range=(5,5), min_df=60, max_df=300)


clf = LogisticRegression()
pipel = Pipeline([('bow', bow),
                 ('classifier', clf)])

pipel.fit(X_train, y_train)
print("Accuracy Score:",pipel.score(X_test, y_test))
import eli5
features = eli5.show_weights(clf, vec=bow, top=1000)

c:\Users\Mateu\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy Score: 0.8523914238592634


In [14]:
from sklearn import metrics

def print_report(pipe):
    y_actuals = y_test
    y_preds = pipe.predict(X_test)
    report = metrics.classification_report(y_actuals, y_preds)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_actuals, y_preds)))

print_report(pipel)

              precision    recall  f1-score   support

     failure       0.87      0.91      0.89      2460
     success       0.80      0.73      0.76      1178

    accuracy                           0.85      3638
   macro avg       0.84      0.82      0.83      3638
weighted avg       0.85      0.85      0.85      3638

accuracy: 0.852


In [12]:
features

Weight?,Feature
+4.276,wysoki podstawa art 39815 orzec
+3.342,orzec sentencja wyrok art 39815
+3.110,sąd wysoki podstawa art 39816
+2.982,zw art 391 art 108
+2.852,wzgląd podstawa art 39815 orzec
+2.562,art 108 zw art 39821
+2.539,myśl art 108 związek art
+2.063,wzgląd orzec sentencja podstawa art
+2.051,następować skarga kasacyjny mieć uzasadniony
+1.970,podstawa art 108 związek art
